In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [75]:
# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 2070 SUPER


In [76]:
# Load Dataset
file_path = 'winequality-white.csv'
data = pd.read_csv(file_path, sep=';')

# Preprocess Dataset
if 'quality' in data.columns:
    labels = data['quality']
    data = data.drop(columns=['quality'])
else:
    raise ValueError("Kolom 'quality' tidak ditemukan dalam dataset.")

# Encode Labels
y = labels.astype('category').cat.codes.values

# Normalize Features
scaler = MinMaxScaler()
X = scaler.fit_transform(data.values.astype(np.float32))

# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape Data
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

batch_size = 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [77]:
# Define Bidirectional RNN Model
class BiRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, pooling_type):
        super(BiRNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            out = torch.max(out, dim=1).values
        elif self.pooling_type == 'avg':
            out = torch.mean(out, dim=1)
        out = self.fc(out)
        return out

In [78]:
# Training Function
def train_model(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs, early_stop_patience):
    model.to(device)
    best_loss = float('inf')
    patience_counter = 0
    history = []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)

        # Validation Step
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).long()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(test_loader)
        val_accuracy = 100.0 * correct / total
        history.append((epoch, avg_loss, val_loss, val_accuracy))

        scheduler.step(val_loss)

        print(f"Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

        # Early Stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"Epoch {epoch}: Validation loss did not improve. Patience = {patience_counter}/{early_stop_patience}")

        if patience_counter >= early_stop_patience:
            print(f"Early stopping triggered at epoch {epoch}.")
            break

    return history

In [79]:
# Experiment Parameters
hidden_sizes = [32, 64]
pooling_types = ['max', 'avg']
optimizers = {'SGD': optim.SGD, 'RMSprop': optim.RMSprop, 'Adam': optim.Adam}
epochs = [5, 50, 100, 250, 350]
learning_rate = 0.001
num_layers = 1
input_size = X_train.shape[2]
output_size = len(np.unique(y))
early_stop_patience = 10

In [80]:
# Run Experiments
results = []
for hidden_size in hidden_sizes:
    for pooling_type in pooling_types:
        for optimizer_name, optimizer_class in optimizers.items():
            for num_epoch in epochs:
                model = BiRNNModel(input_size, hidden_size, num_layers, output_size, pooling_type)
                criterion = nn.CrossEntropyLoss()
                optimizer = optimizer_class(model.parameters(), lr=learning_rate)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

                print(f"Training: Hidden Size={hidden_size}, Pooling={pooling_type}, Optimizer={optimizer_name}, Epochs={num_epoch}")
                history = train_model(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epoch, early_stop_patience)

                final_epoch, train_loss, val_loss, val_accuracy = history[-1]

                results.append({
                    'Hidden Size': hidden_size,
                    'Pooling': pooling_type,
                    'Optimizer': optimizer_name,
                    'Epochs': num_epoch,
                    'Final Epoch': final_epoch,
                    'Train Loss': train_loss,
                    'Validation Loss': val_loss,
                    'Validation Accuracy': val_accuracy
                })

# Save Results
results_df = pd.DataFrame(results)
results_df.to_csv('final_bidirectional_results.csv', index=False)
print("Results saved to final_bidirectional_results.csv")

Training: Hidden Size=32, Pooling=max, Optimizer=SGD, Epochs=5
Epoch 1/5, Loss: 1.8905, Val Loss: 1.8918, Accuracy: 44.08%
Epoch 2/5, Loss: 1.8878, Val Loss: 1.8891, Accuracy: 44.08%
Epoch 3/5, Loss: 1.8852, Val Loss: 1.8865, Accuracy: 44.08%
Epoch 4/5, Loss: 1.8824, Val Loss: 1.8839, Accuracy: 44.08%
Epoch 5/5, Loss: 1.8801, Val Loss: 1.8813, Accuracy: 44.08%
Training: Hidden Size=32, Pooling=max, Optimizer=SGD, Epochs=50
Epoch 1/50, Loss: 1.9248, Val Loss: 1.9257, Accuracy: 8.78%
Epoch 2/50, Loss: 1.9227, Val Loss: 1.9235, Accuracy: 9.80%
Epoch 3/50, Loss: 1.9203, Val Loss: 1.9212, Accuracy: 11.33%
Epoch 4/50, Loss: 1.9182, Val Loss: 1.9189, Accuracy: 13.67%
Epoch 5/50, Loss: 1.9157, Val Loss: 1.9167, Accuracy: 14.69%
Epoch 6/50, Loss: 1.9135, Val Loss: 1.9144, Accuracy: 15.61%
Epoch 7/50, Loss: 1.9111, Val Loss: 1.9122, Accuracy: 17.04%
Epoch 8/50, Loss: 1.9090, Val Loss: 1.9100, Accuracy: 18.78%
Epoch 9/50, Loss: 1.9068, Val Loss: 1.9078, Accuracy: 20.20%
Epoch 10/50, Loss: 1.9045,